In [1]:
import os
import boto3
from botocore.exceptions import ClientError


In [3]:
INDEX_ID = "07c93565-dba9-4902-8077-673c1c09e241"
REGION   = os.environ.get("AWS_REGION", "us-east-1")

def kendra_client(
    region=REGION,
    *,
    access_key: str | None = None,
    secret_key: str | None = None,
    session_token: str | None = None,
    profile: str | None = None,
    role_arn: str | None = None,
):
    """
    Returns a boto3 Kendra client.

    Credential resolution (first match wins):
    1) Explicit long-lived keys passed as args (access_key/secret_key[/session_token])
    2) Named AWS profile (profile)
    3) Default boto3 chain (env vars, ~/.aws/credentials, EC2/ECS role, etc.)
    4) Assume role (role_arn)  -> STS temporary creds

    Notes:
    - For *permanent method*, pass access_key + secret_key, or set them via the AWS
      standard mechanisms (env vars / ~/.aws/credentials).
    - If role_arn is provided, it will be used only when no explicit keys or profile are given.
    """

    # --- 1) Explicit key pair (permanent or temp) ---
    if access_key and secret_key:
        return boto3.client(
            "kendra",
            region_name=region,
            aws_access_key_id=access_key,
            aws_secret_access_key=secret_key,
            # aws_session_token=session_token,  # usually None for long-lived keys
        )

    # --- 2) Specific AWS profile ---
    if profile:
        session = boto3.Session(profile_name=profile)
        return session.client("kendra", region_name=region)

    # --- 3) Default chain (env vars / shared config / instance role) ---
    # If your long-lived keys are in env vars or ~/.aws/credentials, this covers it.
    session = boto3.Session()
    if not role_arn:
        return session.client("kendra", region_name=region)

    # --- 4) Assume role (fallback if requested) ---
    sts = session.client("sts", region_name=region)
    creds = sts.assume_role(RoleArn=role_arn, RoleSessionName="kendra-session")["Credentials"]
    return boto3.client(
        "kendra",
        region_name=region,
        aws_access_key_id=creds["AccessKeyId"],
        aws_secret_access_key=creds["SecretAccessKey"],
        aws_session_token=creds["SessionToken"],
    )

client = kendra_client(
    # Option A (Permanent): pass your long-lived keys directly (recommended only if you must):
    access_key=os.getenv("AWS_ACCESS_KEY_ID"),
    secret_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
)

try:
    info = client.describe_index(Id=INDEX_ID)
    status  = info.get("Status")
    name    = info.get("Name")
    edition = info.get("Edition")
    print(f"Connected to Kendra index: {name} ({INDEX_ID}), edition={edition}, status={status}")
except ClientError as e:
    print(f"DescribeIndex failed: {e.response.get('Error', {}).get('Message', str(e))}")
    raise


Connected to Kendra index: ndIndex (07c93565-dba9-4902-8077-673c1c09e241), edition=GEN_AI_ENTERPRISE_EDITION, status=ACTIVE


In [5]:
def run_query(query_text, page_size=10, query_filter=None, user_groups=None):
    """
    Runs Kendra Query and prints top results (documents + answers).
    - query_filter: optional AttributeFilter dict (e.g., filter by data source, date, etc.)
    - user_groups: list of group strings for ACL-aware search (if you’re using access control)
    """
    args = {
        "IndexId": INDEX_ID,
        "QueryText": query_text,
        "PageSize": page_size,
    }

    if query_filter:
        args["AttributeFilter"] = query_filter

    if user_groups:
        args["UserContext"] = {"Groups": user_groups}

    try:
        resp = client.query(**args)
        print("RES",resp)
    except ClientError as e:
        print(f"Query failed: {e.response.get('Error', {}).get('Message', str(e))}")
        return

    # Extract and print “Answer” and “Document” results
    for item in resp.get("ResultItems", []):
        rtype = item.get("Type")  # "ANSWER" | "DOCUMENT" | "QUESTION_ANSWER" | "TABLE"
        score = item.get("ScoreAttributes", {}).get("ScoreConfidence")
        if rtype in ("ANSWER", "QUESTION_ANSWER"):
            text = item.get("AdditionalAttributes", [])
            answer_text = None
            for aa in text:
                if aa.get("Key") == "AnswerText":
                    answer_text = aa.get("Value", {}).get("TextWithHighlightsValue", {}).get("Text")
            print(f"[{rtype}] ({score}) {answer_text}")
        elif rtype == "DOCUMENT":
            doc = item.get("DocumentTitle", {}).get("Text") or "(untitled)"
            uri = item.get("DocumentURI")
            # Optional: fields such as SourceAttribution and DocumentAttributes:
            print(f"[DOC]   ({score}) {doc} — {uri}")

# Example call
run_query("How much is the book transfer credit for a new business policy?", page_size=5)


RES {'QueryId': '2bf2a87f-0e87-43e5-9147-4355ccfcdb67', 'ResultItems': [{'Id': '2bf2a87f-0e87-43e5-9147-4355ccfcdb67-1154761e-8a15-4298-8573-b3cc4d638096', 'Type': 'ANSWER', 'Format': 'TEXT', 'AdditionalAttributes': [{'Key': 'AnswerText', 'ValueType': 'TEXT_WITH_HIGHLIGHTS_VALUE', 'Value': {'TextWithHighlightsValue': {'Text': '\nDorchester Mutual Insurance Companies. \n\n\nEndorsement: HO-LF \n\n\n  \n\n\n\n\n\n\n\nNJ PERSONAL LINES MANUAL \n\n\nFITCHBURG MUTUAL INSURANCE CO.  HO • R 9 \n\n\n \n\n\n \nEd. 5.0 \n\n\n© 2017 MSO®, Inc. \n\n\n24.0  BOOK TRANSFER CREDIT \n\n\nA 10% credit applies to new business that is part of an agreed book transfer from a designated company in \n\n\nthe agent’s office. Credit applies to new business and to the next 4 renewals as follows: \n\n\n Credit % \nNew Business 10% \n\n\n1st renewal 8% \n\n\n2nd renewal 6% \n\n\n3rd renewal 4% \n\n\n4th renewal 2% \n\n\n5th renewal deleted \nEndorsement: BTC 09 \n\n\nCannot be applied concurrently with the Agency 